In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module): #nn.Module 모든 신경망 모듈의 기본이 되는 클래스
                      #각 층과 함수 등 신경망의 구성요소를 이 클래스 안에서 정의
    def __init__(self):
        super(Net, self).__init__() #부모 클래스 상속
        # 1 input image channel, 6 output channels, 3x3 square convolution(=filter)
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3) #1 channel
        self.conv2 = nn.Conv2d(6, 16, 3) #입력 채널 수, 출력 채널 수, 필터크기, (stride=1, padding=0)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)  # 6*6 from image dimension
        # 입력 샘플 크기, 출력 샘플 크기
        #16*6*6 : conv2 레이어를 지난 출력 채널의 수=16, 입력 이미지의 dimension=6*6
        #nn.Linear함수 : 선형 함수를 사용해 입력으로부터 출력 계산, 내부 tensor에 가중치와 편향 저장
        self.fc2 = nn.Linear(120, 84) #fc1의 출력 -> fc2의 입력
        self.fc3 = nn.Linear(84, 10) #fc2의 출력 -> fc3의 출력

    #x를 매개변수로 forward 함수를 정의
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) #conv1 계산 -> max pooling
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) #conv2 계산 -> max pooling
        x = x.view(-1, self.num_flat_features(x)) # '.view()': tensor의 크기와 모양을 변경
                                                  #num_flat_features(self,x) 함수 하단에 정의
        x = F.relu(self.fc1(x)) #fc1 레이어 -> relu 계산(activation func)
        #활성화 함수 : node에 들어오는 값들에 대해 비선형 함수를 통과시킨 후 다음 레이어로 전달
        #활성화 함수 = 비선형 함수
        #선형 함수인 h(x)=cx 를 활성화 함수로 사용한 3층 네트워크를 생각해보자
        #y(x)=h(h(h(x)))로 결국 y(x)=ax와 똑같은 식 -> 은닉층이 없는 네트워크
        #NN에서 층을 쌓고 싶다면 비선형 함수를 이용
        x = F.relu(self.fc2(x)) #fc2 레이어 -> relu 계산
        x = self.fc3(x) #최종 레이어이므로 활성화 함수 사용x
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s #size라는 목록의 각각의 항목 s에 대하여 계산
        return num_features


net = Net() #임의로 실행
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수만 정의하고 나면, `backward`함수는 `autograd`를 사용해 자동으로 정의

모델의 학습 가능한 매개변수들(=가중치)은 `net.parameters()`에 의해 반환

In [3]:
params=list(net.parameters()) #매개변수 리스트 생성
print(len(params))
print(params[0].size()) #conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input=torch.randn(1,1,32,32)
out=net(input)
print(out) #최종 fc3 outpt sample size = 10

tensor([[-0.0461, -0.0539, -0.0117, -0.1220, -0.0539,  0.0183, -0.0419,  0.0354,
          0.0384, -0.0164]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad() #모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정
out.backward(torch.randn(1,10)) #무작위 값으로 역전파

#### `torch.nn`은 미니배치만 지원, 샘플들의 미니배치만을 입력으로 받음
#### `nnConv2D` 는 nSamples x nChannels x Height x Width
#### 만약 하나의 샘플만 있다면, `input.unsqueeze(0)` 을 사용해 가짜 차원을 추가



> 요약

`torch.Tensor` - `backward()` 같은 autograd 연산을 지원하는 다차원 배열

또한 tensor에 대한 변화도(gradient)를 가짐

`nn.Module` - 신경망 모듈. 

매개변수를 캡슐화 (gpu로 이동, 내보내기, 불러오기 등의 작업을 위한 헬퍼를 제공)

`nn.Parameter` - tensor의 한 종류로, Module에 속성으로 할당될 때 자동으로 매개변수 등록

`autograd.Function` - autograd 연산의 전방향과 역방향 정의를 구현


---

### 손실함수 (loss function) 
 : 한 쌍의 입력으로 받아 출력이 target으로부터 얼마나 멀리 떨어져 있는지 추정하는 값

In [6]:
#평균제곱오차 - nn.MSEloss
output=net(input)
target=torch.randn(10) #a dummy target, for ex
target=target.view(1,-1) #make it the same shape as output
# -1 은 차원 수 보고 알맞게 알아서 설정하라는 의미
criterion=nn.MSELoss()

loss=criterion(output,target)
print(loss)

tensor(0.9435, grad_fn=<MseLossBackward>)


`loss.backward()`를 실행할 때, 전체 그래프는 손실에 대하여 미분되며,

그래프 내의 `requires_Grad=True` 인 모든 tensor는 gradient가 누적된 `.grad` tensor를 가짐

In [7]:
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) #linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU
#각 단계의 gradients

---
### 역전파 (backprop)
오차를 역전파하기 위해서는 `loss.backward()` 만 해주면 됨

변화도가 기존의 것에 누적될 수 있기 때문에 기존 변화도를 없애는 작업 필요


In [8]:
net.zero_grad() #zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0135, -0.0126,  0.0012,  0.0125,  0.0189, -0.0043])


---
### 가중치 갱신
많이 사용되는 단순한 갱신 규칙 : 확률적 경사하강법 SGD

가중치=가중치-학습율*변화도

In [9]:
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

신경망 구성 시 다양한 갱신 규칙을 사용하기 위해 `torch.optim` 에 구현되어 있음

In [ ]:
import torch.optim as optim

#optimizer를 생성
optimizer=optim.SGD(net.parameters(),lr=0.01)
#SGD를 직접 정의하지 않고, torch.optim 패키지 속 모듈 사용

#학습과정(training loop) :
optimizer.zero_grad() #zero the gradient buffers
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step() #does the update
